<a href="https://colab.research.google.com/github/dine1717/END2/blob/Session5/StanfordSentiment_Analysis_using_LSTM_RNN_S5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

In [ ]:
!nvidia-smi

Thu Jun  3 17:18:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Colab\ Files/stanfordSentimentTreebank.zip

Archive:  /content/drive/MyDrive/Colab Files/stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: __MACOSX/
   creating: __MACOSX/stanfordSentimentTreebank/
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSentences.txt  
  inflating: stanfordSentimentTreebank/datasetSplit.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSplit.txt  
  inflating: stanfordSentimentTreebank/dictionary.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._dictionary.txt  
  inflating: stanfordSentimentTreebank/original_rt_snippets.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._original_rt_snippets.txt  
  inflating: stanfordSentimentTreebank/README.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._README.txt  
  inflating: stanfordSentimentTreebank/sentiment_labels.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._sentiment_labels.txt  
  inflating: stanfordSentimentTreeb

In [ ]:
!ls stanfordSentimentTreebank

datasetSentences.txt  dictionary.txt		README.txt	      SOStr.txt
datasetSplit.txt      original_rt_snippets.txt	sentiment_labels.txt  STree.txt


In [ ]:
"""
Put all the Stanford Sentiment Treebank phrase data into test, training, and dev CSVs.

Socher, R., Perelygin, A., Wu, J. Y., Chuang, J., Manning, C. D., Ng, A. Y., & Potts, C. (2013). Recursive Deep Models
for Semantic Compositionality Over a Sentiment Treebank. Presented at the Conference on Empirical Methods in Natural
Language Processing EMNLP.

https://nlp.stanford.edu/sentiment/
"""

import os
import sys

import pandas


def get_phrase_sentiments(base_directory):


    dictionary = pandas.read_csv(os.path.join(base_directory, "dictionary.txt"), sep="|")
    dictionary.columns = ["phrase", "id"]
    dictionary = dictionary.set_index("id")

    sentiment_labels = pandas.read_csv(os.path.join(base_directory, "sentiment_labels.txt"), sep="|")
    sentiment_labels.columns = ["id", "sentiment"]
    sentiment_labels = sentiment_labels.set_index("id")

    phrase_sentiments = dictionary.join(sentiment_labels)

    phrase_sentiments["fine"] = pandas.cut(phrase_sentiments.sentiment, [0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                           include_lowest=True,
                                           labels=["very negative", "negative", "neutral", "positive", "very positive"])
    #phrase_sentiments["coarse"] = phrase_sentiments.fine.apply(group_labels)
    return phrase_sentiments


def get_sentence_partitions(base_directory):
    sentences = pandas.read_csv(os.path.join(base_directory, "datasetSentences.txt"), index_col="sentence_index",
                                sep="\t")
    splits = pandas.read_csv(os.path.join(base_directory, "datasetSplit.txt"), index_col="sentence_index")
    return sentences.join(splits).set_index("sentence")


def partition(base_directory):
    phrase_sentiments = get_phrase_sentiments(base_directory)
    sentence_partitions = get_sentence_partitions(base_directory)
    # noinspection PyUnresolvedReferences
    data = phrase_sentiments.join(sentence_partitions, on="phrase")
    data["splitset_label"] = data["splitset_label"].fillna(1).astype(int)
    data["phrase"] = data["phrase"].str.replace(r"\s('s|'d|'re|'ll|'m|'ve|n't)\b", lambda m: m.group(1))
    return data.groupby("splitset_label")


base_directory, output_directory = "stanfordSentimentTreebank","."
os.makedirs(output_directory, exist_ok=True)
for splitset, partition in partition(base_directory):
    split_name = {1: "train", 2: "test", 3: "dev"}[splitset]
    filename = os.path.join(output_directory, "stanford-sentiment-treebank.%s.csv" % split_name)
    del partition["splitset_label"]
    partition.to_csv(filename)

In [ ]:
!ls

drive			   stanford-sentiment-treebank.dev.csv
__MACOSX		   stanford-sentiment-treebank.test.csv
sample_data		   stanford-sentiment-treebank.train.csv
stanfordSentimentTreebank


## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [ ]:
import pandas as pd
train_df = pd.read_csv('stanford-sentiment-treebank.train.csv')
train_df.head()

,id,phrase,sentiment,fine
0,22935,! ',0.52778,neutral
1,18235,! '',0.50000,neutral
2,179257,! Alas,0.44444,neutral
3,22936,! Brilliant,0.86111,very positive
4,40532,! Brilliant !,0.93056,very positive


In [ ]:
train_df.shape

(236076, 4)

In [ ]:
train_df.fine.value_counts()

neutral          118856
positive          49402
negative          42153
very positive     14713
very negative     10952
Name: fine, dtype: int64

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    # if len(words) == 1: # return if single word
    #     return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
!pip3 install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 1.0MB 31.6MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=0f55aa7a67e9aad95251183f6eaebcebd2ac85a3e634659c9300b9e200159718
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [ ]:
import random
import googletrans
from googletrans import Translator

# import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)

Translating to thai
['หมานอนบนพรม']
['dog sleeping on carpet']


In [ ]:
def back_translate(sentence,trans_lang):
  translator = Translator()
  translations = translator.translate(sentence, dest=trans_lang)
  #t_text = [t.text for t in translations]

  translations_en_random = translator.translate(translations.text, src=trans_lang, dest='en') 
  #en_text = [t.text for t in translations_en_random]
  return translations_en_random.text


In [ ]:
# available_langs = list(googletrans.LANGUAGES.keys()) 
# trans_lang = random.choice(available_langs)
translate_df= train_df[train_df['phrase'].str.count('\s+')>10].sample(n=100,random_state=43)
translate_df['phrase'] = translate_df['phrase'].apply(back_translate,trans_lang=trans_lang)

In [ ]:
random_deletion_df = train_df[train_df['phrase'].str.count('\s+')>10].sample(frac=0.5)
random_deletion_df.head(5)

,id,phrase,sentiment,fine
173681,9826,"pretension , in its own way , is a form of bra...",0.69444,positive
49563,69548,The film has the uncanny ability to right itse...,0.75000,positive
84913,75498,begins in Saigon in 1952 . That's its first si...,0.33333,negative
87439,29723,brave attempt to tap into the heartbeat of the...,0.76389,positive
59744,14672,a blandly cinematic surgical examination of wh...,0.37500,negative


In [ ]:
random_deletion_df['phrase'] = random_deletion_df.phrase.str.split().apply(random_deletion).apply(' '.join)
random_deletion_df.head(5)

,id,phrase,sentiment,fine
173681,9826,pretension a form bravery for this reason -- p...,0.69444,positive
49563,69548,The film ability when think it's in of going w...,0.75000,positive
84913,75498,in in That's its sign,0.33333,negative
87439,29723,"brave attempt to tap the heartbeat the , a sal...",0.76389,positive
59744,14672,cinematic surgical examination makes a joke,0.37500,negative


In [ ]:
random_swap_df = train_df[train_df['phrase'].str.count('\s+')>10].sample(frac=0.5)
random_swap_df.head(5)


,id,phrase,sentiment,fine
4788,61389,"'ve marveled at Disney's rendering of water , ...",0.84722,very positive
18006,104252,An already thin story boils down to surviving ...,0.12500,very negative
199695,173502,the awkward interplay and utter lack of chemis...,0.20833,negative
57811,151096,"Yes , one enjoys seeing Joan grow from awkward...",0.44444,neutral
123896,201129,his ( Nelson's ) screenplay needs some serious...,0.30556,negative


In [ ]:
random_swap_df['phrase'] = random_swap_df.phrase.str.split().apply(random_swap).apply(' '.join)
random_swap_df.head(5)


,id,phrase,sentiment,fine
4788,61389,"'ve marveled at Disney's rendering water to , ...",0.84722,very positive
18006,104252,An already seeking story boils down to survivi...,0.12500,very negative
199695,173502,of lack the Chan utter awkward between chemist...,0.20833,negative
57811,151096,"Yes , grow enjoys awkward , one from seeing yo...",0.44444,neutral
123896,201129,more ( Nelson's ) screenplay some show serious...,0.30556,negative


In [ ]:
final_train_df = pd.concat([train_df,random_deletion_df,random_swap_df,translate_df], axis=0).reset_index(drop=True)
#final_train_df['coarse'].replace(['negative','positive','neutral'],[0,1,2],inplace=True)
#final_train_df = pd.concat([train_df,random_deletion_df,random_swap_df], axis=0).reset_index(drop=True)
final_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285630 entries, 0 to 285629
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         285630 non-null  int64  
 1   phrase     285630 non-null  object 
 2   sentiment  285630 non-null  float64
 3   fine       285630 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 8.7+ MB


In [ ]:
final_train_df.head()

,id,phrase,sentiment,fine
0,22935,! ',0.52778,neutral
1,18235,! '',0.50000,neutral
2,179257,! Alas,0.44444,neutral
3,22936,! Brilliant,0.86111,very positive
4,40532,! Brilliant !,0.93056,very positive


In [ ]:
test_df = pd.read_csv('stanford-sentiment-treebank.test.csv')
#test_df['coarse'].replace(['negative','positive','neutral'],[0,1,2],inplace=True)

valid_df = pd.read_csv('stanford-sentiment-treebank.dev.csv')
#valid_df['coarse'].replace(['negative','positive','neutral'],[0,1,2],inplace=True)



## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Phrase =data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [ ]:
fields = [('phrase', Phrase),('coarse',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [ ]:
train_example = [data.Example.fromlist([final_train_df.phrase[i],final_train_df.fine[i]], fields) for i in range(final_train_df.shape[0])] 
valid_example = [data.Example.fromlist([valid_df.phrase[i],valid_df.fine[i]], fields) for i in range(valid_df.shape[0])] 
test_example = [data.Example.fromlist([test_df.phrase[i],test_df.fine[i]], fields) for i in range(test_df.shape[0])] 

In [ ]:
# Creating dataset
train = data.Dataset(train_example, fields)
valid = data.Dataset(valid_example, fields)
test = data.Dataset(test_example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(len(train), len(valid))

(285630, 1044)

An example from the dataset:

In [ ]:
train.examples[1007].__dict__

{'coarse': 'negative',
 'phrase': ["'re",
  'burnt',
  'out',
  'on',
  'It',
  "'s",
  'a',
  'Wonderful',
  'Life',
  'marathons',
  'and',
  'bored',
  'with',
  'A',
  'Christmas',
  'Carol']}

In [ ]:
vars(train.examples[100]).get('phrase')

["'", 'a', 'nightmare', 'on', 'elm', 'street', "'"]

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [ ]:
Phrase.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [ ]:
print('Size of input vocab : ', len(Phrase.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Phrase.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  20896
Size of label vocab :  3
Top 10 words appreared repeatedly : [('the', 105388), (',', 100013), ('a', 74826), ('of', 72415), ('and', 70247), ('to', 52839), ('.', 52488), ('-', 46418), ("'s", 37698), ('is', 31732)]
Labels :  defaultdict(<function _default_unk_index at 0x7f3323724048>, {'neutral': 0, 'positive': 1, 'negative': 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_iterator, valid_iterator,test_iterator = data.BucketIterator.splits((train, valid,test), batch_size = 32, 
                                                            sort_key = lambda x: len(x.phrase),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Phrase.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features - this is basically the size of hidden state/context vector the dimension of both context/hidden vector will be same (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with five outputs to correspond to our five classes 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Phrase.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(20896, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 6,510,703 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
for batch in train_iterator:
  print(batch.phrase)
  break

(tensor([[    4,   155,  1567,   987,    35, 14371,    10,  3602],
        [   11,    16,   116,    49,    10,    47,    23, 15187],
        [    7,  8241,  1736,     8,    28,    64,   117,   630],
        [  308,    58,   568,  1726, 11880,    32,    35, 11798],
        [   18,   105,     9,   467,     5,   186,     6,  1313],
        [    7,  5506,   999,   128,   193,    35,   411,   743],
        [18390,     2,  3283,  1839,   163,    24,  4656,  1081],
        [    5,   787,  1180,    19,     4,     5,   137,  3629],
        [   61,     6,    29,  1720,  1730,  3538, 13776,    29],
        [  227,    30,   278,    15,    62,    17,     2,    53],
        [    4,    92,    60,    20,    12,    90,  2627,   156],
        [ 2680,     3,   959,  2027,     6,  1987,     3,     6],
        [  149,    24,  6489,    16,  4022,    33,  1719,     8],
        [  120,    63,    11,    30,    62,     7,   449,    38],
        [ 6874,  2387,  3546,    13,   635,   135,  2353,     6],
        [

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sentence, phrase_lengths = batch.phrase  
        
        # convert to 1D tensor
        predictions = model(sentence, phrase_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.coarse)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.coarse)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, phrase_lengths = batch.phrase
            
            # convert to 1d tensor
            predictions = model(sentence, phrase_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.coarse)
            acc = binary_accuracy(predictions, batch.coarse)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.933 | Train Acc: 61.09%
	 Val. Loss: 0.864 |  Val. Acc: 67.80% 

	Train Loss: 0.835 | Train Acc: 71.13%
	 Val. Loss: 0.838 |  Val. Acc: 70.32% 

	Train Loss: 0.801 | Train Acc: 74.69%
	 Val. Loss: 0.817 |  Val. Acc: 73.30% 

	Train Loss: 0.781 | Train Acc: 76.79%
	 Val. Loss: 0.818 |  Val. Acc: 72.67% 

	Train Loss: 0.767 | Train Acc: 78.27%
	 Val. Loss: 0.811 |  Val. Acc: 73.81% 

	Train Loss: 0.755 | Train Acc: 79.51%
	 Val. Loss: 0.809 |  Val. Acc: 73.77% 

	Train Loss: 0.746 | Train Acc: 80.46%
	 Val. Loss: 0.810 |  Val. Acc: 73.81% 

	Train Loss: 0.738 | Train Acc: 81.30%
	 Val. Loss: 0.803 |  Val. Acc: 74.34% 

	Train Loss: 0.731 | Train Acc: 82.04%
	 Val. Loss: 0.814 |  Val. Acc: 73.39% 

	Train Loss: 0.725 | Train Acc: 82.66%
	 Val. Loss: 0.817 |  Val. Acc: 73.11% 



## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.777 | Test Acc: 76.78%


In [ ]:
0#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentiment(sentence):
    
    categories = {2: "Negative", 1:"Positive", 0:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)
 
    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_sentiment("A valid explanation for why Trump won't let women on the golf course.")

'Negative'

In [ ]:
classify_sentiment("I feel great this morning.")

'Positive'